In [1]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel
from scipy.io import loadmat
import pickle
import numpy as np
from pgmpy.inference import VariableElimination
from pgmpy.sampling import GibbsSampling#BayesianModelSampling

# Setting up your model

### First, set the structure

In [2]:
s = loadmat('./structure.mat') 
dag = np.array(s['dag'])
vari_card = np.array(s['domain_counts'][0])
with open('./parameter.pkl', 'rb') as fp:
    CPTs = pickle.load(fp)
variable_name = ['BirthAsphyxia', 'Disease', 'Age', 'LVH', 'DuctFlow', 'CardiacMixing',\
                 'LungParench','LungFlow', 'Sick', 'HypDistrib', 'HypoxiaInO2', 'CO2', \
                 'ChestXray', 'Grunting', 'LVHreport', 'LowerBodyO2','RUQO2', 'CO2Report', 'XrayReport', 'GruntingReport']
child_model = BayesianModel()
child_model.add_nodes_from(variable_name) 
row, col = np.shape(dag)
pai_all = []
pai_card_all = []
for i in range(col):
    pai =[];
    pai_card =[];
    for j in range(row):
        if dag[j,i] == 1:
            child_model.add_edges_from([(variable_name[j], variable_name[i]) ])
            pai.append(variable_name[j])
            pai_card.append(vari_card[j])
    pai_all.append(pai)
    pai_card_all.append(pai_card)
print (pai_all)
print (pai_card_all)

[[], ['BirthAsphyxia'], ['Disease', 'Sick'], ['Disease'], ['Disease'], ['Disease'], ['Disease'], ['Disease'], ['Disease'], ['DuctFlow', 'CardiacMixing'], ['CardiacMixing', 'LungParench'], ['LungParench'], ['LungParench', 'LungFlow'], ['LungParench', 'Sick'], ['LVH'], ['HypDistrib', 'HypoxiaInO2'], ['HypoxiaInO2'], ['CO2'], ['ChestXray'], ['Grunting']]
[[], [2], [6, 2], [6], [6], [6], [6], [6], [6], [3, 4], [4, 3], [3], [3, 3], [3, 2], [2], [2, 3], [3], [3], [5], [2]]


### Then set up the relationships (the CPDs), Add the relationships to your models

In [3]:
for i in range(len(variable_name)): 
    cpd = TabularCPD(
           variable  = variable_name[i],
           variable_card = vari_card[i],
           values =  CPTs[variable_name[i]],
           evidence = pai_all[i],
           evidence_card = pai_card_all[i] ) 
    child_model.add_cpds(cpd) 

### Examine the structure of your graph

In [4]:
child_model.get_cpds()

[<TabularCPD representing P(BirthAsphyxia:2) at 0x17dbe748>,
 <TabularCPD representing P(Disease:6 | BirthAsphyxia:2) at 0x17dbe6d8>,
 <TabularCPD representing P(Age:3 | Disease:6, Sick:2) at 0x17dbe630>,
 <TabularCPD representing P(LVH:2 | Disease:6) at 0x17dbe668>,
 <TabularCPD representing P(DuctFlow:3 | Disease:6) at 0x17dabd30>,
 <TabularCPD representing P(CardiacMixing:4 | Disease:6) at 0x17dabd68>,
 <TabularCPD representing P(LungParench:3 | Disease:6) at 0x17dbe780>,
 <TabularCPD representing P(LungFlow:3 | Disease:6) at 0x17dbe7b8>,
 <TabularCPD representing P(Sick:2 | Disease:6) at 0x17dbe860>,
 <TabularCPD representing P(HypDistrib:2 | DuctFlow:3, CardiacMixing:4) at 0x17dbe8d0>,
 <TabularCPD representing P(HypoxiaInO2:3 | CardiacMixing:4, LungParench:3) at 0x17dbe908>,
 <TabularCPD representing P(CO2:3 | LungParench:3) at 0x17dbe898>,
 <TabularCPD representing P(ChestXray:5 | LungParench:3, LungFlow:3) at 0x17dbe7f0>,
 <TabularCPD representing P(Grunting:2 | LungParench:3, 

In [5]:
child_model.active_trail_nodes('Disease')

{'Disease': {'Age',
  'BirthAsphyxia',
  'CO2',
  'CO2Report',
  'CardiacMixing',
  'ChestXray',
  'Disease',
  'DuctFlow',
  'Grunting',
  'GruntingReport',
  'HypDistrib',
  'HypoxiaInO2',
  'LVH',
  'LVHreport',
  'LowerBodyO2',
  'LungFlow',
  'LungParench',
  'RUQO2',
  'Sick',
  'XrayReport'}}

# Making inferences

In [6]:
child_infer = VariableElimination(child_model) 

### We can also get conditional probability distributions that take into account what we already know

In [7]:
prob_BirthAsphyxia_evidence = child_infer.query(
                                        variables = ['BirthAsphyxia'],
                                        evidence = {'CO2Report': 0, 'LVHreport':0, 'XrayReport': 2})
print(prob_BirthAsphyxia_evidence['BirthAsphyxia']) 
prob_Disease_evidence = child_infer.query(
                                        variables = ['Disease'],
                                        evidence = {'CO2Report': 0, 'LVHreport':0, 'XrayReport': 2})
print(prob_Disease_evidence['Disease']) 

╒═════════════════╤══════════════════════╕
│ BirthAsphyxia   │   phi(BirthAsphyxia) │
╞═════════════════╪══════════════════════╡
│ BirthAsphyxia_0 │               0.0813 │
├─────────────────┼──────────────────────┤
│ BirthAsphyxia_1 │               0.9187 │
╘═════════════════╧══════════════════════╛
╒═══════════╤════════════════╕
│ Disease   │   phi(Disease) │
╞═══════════╪════════════════╡
│ Disease_0 │         0.0235 │
├───────────┼────────────────┤
│ Disease_1 │         0.1077 │
├───────────┼────────────────┤
│ Disease_2 │         0.1429 │
├───────────┼────────────────┤
│ Disease_3 │         0.6834 │
├───────────┼────────────────┤
│ Disease_4 │         0.0103 │
├───────────┼────────────────┤
│ Disease_5 │         0.0322 │
╘═══════════╧════════════════╛


### We can get probability distributions that are not explicitly spelled out in our graphs

In [13]:
prob_BirthAsphyxia = child_infer.query(variables = ['BirthAsphyxia'])
print(prob_BirthAsphyxia['BirthAsphyxia'])

### We can find out the most probable state for a variable

In [14]:
child_infer.map_query(variables = ['Disease'],
                      evidence = {'CO2Report': 0, 'LVHreport':0, 'XrayReport': 2})

{'Disease': 3}

### Find independencies

In [62]:
#child_model.local_independencies('Age')
#child_model.get_independencies()

(Age _|_ CO2, Grunting, LVHreport, BirthAsphyxia, GruntingReport, ChestXray, HypDistrib, DuctFlow, LVH, HypoxiaInO2, CO2Report, RUQO2, LungFlow, LungParench, CardiacMixing, LowerBodyO2, XrayReport | Disease, Sick)

In [14]:
#child_infer = GibbsSampling(child_model)
#Gibbs_results = child_infer.sample(size=100000, return_type='recarray')
#print (type(Gibbs_results))
#print (np.shape(Gibbs_results))

KeyboardInterrupt: 